<a href="https://colab.research.google.com/github/deeev2005/Ghost-GPT/blob/main/Deepfake_Detector_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#---------- Uploading CRED ---------
from google.colab import files
files.upload()  # Select kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"deeev2005","key":"da96a327d391108056acdd3f9f03d5e6"}'}

In [7]:
#--------------- Installing necessary libraries -----------
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
!kaggle datasets download -d birdy654/deep-voice-deepfake-voice-recognition
!unzip deep-voice-deepfake-voice-recognition.zip -d deep_voice_data



Dataset URL: https://www.kaggle.com/datasets/birdy654/deep-voice-deepfake-voice-recognition
License(s): other
 99% 3.66G/3.69G [00:36<00:00, 146MB/s]
100% 3.69G/3.69G [00:36<00:00, 108MB/s]
Archive:  deep-voice-deepfake-voice-recognition.zip
  inflating: deep_voice_data/DEMONSTRATION/DEMONSTRATION/linus-original-DEMO.mp3  
  inflating: deep_voice_data/DEMONSTRATION/DEMONSTRATION/linus-to-musk-DEMO.mp3  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/Obama-to-Biden.wav  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/Obama-to-Trump.wav  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/biden-to-Obama.wav  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/biden-to-Trump.wav  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/biden-to-linus.wav  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/biden-to-margot.wav  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/biden-to-musk.wav  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/biden-to-ryan.wav  
  inflating: deep_voice_data/KAGGLE/AUDIO/FAKE/bid

In [9]:
#---------- Importing ---------
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from tqdm import tqdm


In [10]:
#--------------- loading dataset from kaggle ---------
real_dir = 'deep_voice_data/KAGGLE/AUDIO/REAL'
fake_dir = 'deep_voice_data/KAGGLE/AUDIO/FAKE'


output_dir = 'mel_spectrograms'
os.makedirs(f"{output_dir}/real", exist_ok=True)
os.makedirs(f"{output_dir}/fake", exist_ok=True)


In [11]:
#----------- Converting audio files in spectogram imsges function -----------
def create_mel_spectrogram(audio_path, output_path, sr=22050, n_mels=128):
    try:
        y, sr = librosa.load(audio_path, sr=sr)
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        mel_db = librosa.power_to_db(mel_spec, ref=np.max)

        plt.figure(figsize=(3, 3))
        librosa.display.specshow(mel_db, sr=sr, cmap='magma')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close()
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")


In [12]:
def process_folder(input_folder, output_folder):
    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith(".wav"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename.replace(".wav", ".png"))
            create_mel_spectrogram(input_path, output_path)

process_folder(real_dir, f"{output_dir}/real")
process_folder(fake_dir, f"{output_dir}/fake")


100%|██████████| 56/56 [05:13<00:00,  5.61s/it]


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [14]:
data_dir = 'mel_spectrograms'

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [15]:
#------------ Binary classification ----------
batch_size = 32

train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

class_names = train_dataset.classes  # ['fake', 'real']
print("Classes:", class_names)


Classes: ['fake', 'real']


In [16]:
from torchvision import models
import torch.nn as nn

# Load pretrained ResNet-50
resnet50 = models.resnet50(pretrained=True)

# Freeze early layers (optional)
for param in resnet50.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, 2)  # Binary classification

# Move to device (GPU if available) for faster computing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet50.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 149MB/s]


In [17]:
#------------parameterizing ------------
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [18]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {running_loss/len(train_loader):.4f} | Accuracy: {acc:.2f}%")


Epoch 1/10 | Loss: 0.7618 | Accuracy: 39.06%
Epoch 2/10 | Loss: 0.6498 | Accuracy: 60.94%
Epoch 3/10 | Loss: 0.5731 | Accuracy: 82.81%
Epoch 4/10 | Loss: 0.5103 | Accuracy: 85.94%
Epoch 5/10 | Loss: 0.4654 | Accuracy: 87.50%
Epoch 6/10 | Loss: 0.4553 | Accuracy: 87.50%
Epoch 7/10 | Loss: 0.4370 | Accuracy: 87.50%
Epoch 8/10 | Loss: 0.4388 | Accuracy: 87.50%
Epoch 9/10 | Loss: 0.4294 | Accuracy: 87.50%
Epoch 10/10 | Loss: 0.4397 | Accuracy: 87.50%
